# Análisis preliminar del conjunto de datos

In [ ]:
# Importaciones de paquetes
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
# Importaciones de unidades de soporte
import sys
sys.path.append("..")
from src import sp_eda as sp

In [205]:
df_raw = pd.read_csv("../data/halcon_viajes_data.csv") # Leer el archivo

In [206]:
df = df_raw.copy() # Hacer una copia del dataframe con el que trabajar

In [ ]:
sp.eda_preliminar(df) # Hacer un primer análisis exploratorio de los datos

## Limpieza de datos

In [ ]:
df = df.drop(columns=['dni_usuario']) # Eliminar columnas innecesarias para el análisis

# Normalización de datos
df.columns = df.columns.str.lower()

lista_cat = ['género', 'ubicación','tipo_viajero', 'tipo_paquete','metodo_pago', 'destino',
       'tipo_alojamiento', 'clase_vuelo', 'actividades_reservadas', 'fuente_reserva','estado_reserva', 'estacionalidad',
       'comentarios']
for col in lista_cat:
  df[col] = df[col].str.replace(" ","_").str.lower()

# Convertir los datos a un formato coherente y consistente
df['género'] = df['género'].map({'female':'femenino', 'male':'masculino','other':'otro'})
df['ubicación'] = df['ubicación'].replace({'germany':'alemania', 'spain':'españa'})
df['tipo_viajero'] = df['tipo_viajero'].replace({'tusirta':'turista'})
df['tipo_paquete'] = df['tipo_paquete'].replace({'vuelo_+_hotel_+_tour': 'vuelo_hotel_tour', 'vuelo_+_hotel':'vuelo_hotel' })
df['destino'] = df['destino'].replace({'new_york':'nueva_york','london':'londres','rome':'roma','tokyo':'tokio'})

# Reemplazar True por 'si' y False por 'no' en las columnas booleanas
col_bool = ['promocion_aplicada', 'cancelacion_reserva']
df[col_bool] = df[col_bool].applymap(lambda x: 'si' if x else 'no')

# Convertir las columnas temporales a tipo datetime
for col in ['fecha_reserva', 'fecha_viaje']:
    df[col] = pd.to_datetime(df[col], format='%Y|%m|%d %H:%M:%S', errors='coerce')
    df[col] = df[col].dt.strftime('%Y/%m/%d')
    df[col] = pd.to_datetime(df[col], format='%Y/%m/%d', errors='coerce')

## Validación de datos

In [ ]:
# Validación de fechas: Asegurar que 'fecha_reserva' sea anterior a 'fecha_viaje'.
validacion_fecha = df[df['fecha_reserva'] > df['fecha_viaje']]

# Verificar que la duración del viaje es mayor que el número de noches de estancia
validacion_duracion = df[df['duracion_viaje'] <= df['numero_noches_estancia']]

# Verificar que el valor de 'mes_viaje' se corresponda con el mes de 'fecha_viaje'
validacion_mes_viaje = df[df['mes_viaje'] != df['fecha_viaje'].dt.month]

# Verificar que el valor de 'mes_reserva' se corresponda con el mes de 'fecha_reserva'
validacion_mes_reserva = df[df['mes_reserva'] != df['fecha_reserva'].dt.month]

# Verificar que las columnas 'estado_reserva' y 'cancelacion_reserva' sean coherentes
validacion_estado_cancelacion = df[(df['estado_reserva'] == 'cancelada') & (df['cancelacion_reserva'] != 'si') | (df['estado_reserva'] != 'cancelada') & (df['cancelacion_reserva'] == 'si')]

In [ ]:
# Corregir datos inconsistentes
df['mes_viaje'] = df['fecha_viaje'].dt.month
df['mes_reserva'] = df['fecha_reserva'].dt.month
# Corregir las inconsistencias entre 'estado_reserva' y 'cancelacion_reserva'
df.loc[(df['estado_reserva'] == 'cancelada') & (df['cancelacion_reserva'] != 'si'), 'cancelacion_reserva'] = 'si'
df.loc[(df['estado_reserva'] != 'cancelada') & (df['cancelacion_reserva'] == 'si'), 'cancelacion_reserva'] = 'no'

## Explicaciones de las inconsistencias detectadas en la validación de datos

### 1. Duración del viaje y número de noches de estancia

Durante el proceso de validación de datos, se observó que en **379 registros** de los datos, la **duración del viaje** es **menor o igual** al **número de noches de estancia**. Este comportamiento podría parecer ilógico, ya que se esperaría que la duración total del viaje sea siempre mayor que el número de noches pasadas en el destino.

Sin embargo, después de revisar los datos y el contexto del negocio, se concluyó que **no hay un error en los datos**. La discrepancia puede ser explicada por varias razones legítimas, como:
- Los viajeros pueden haber reservado viajes largos, pero con una estancia corta en el destino, lo que hace que la duración del viaje sea mayor que la estancia en sí.
- La duración del viaje podría incluir tiempos de vuelo, escalas o trayectos que no se cuentan como noches de estancia en el destino.

#### Decisión tomada:
Se decidió **mantener los datos tal como están**. No se realizaron modificaciones a las columnas `duracion_viaje` y `numero_noches_estancia`, ya que el comportamiento observado es plausible dentro del contexto de la gestión de viajes.

#### Impacto:
Este enfoque garantiza que no se eliminen registros ni se modifiquen datos válidos. Además, se preserva la integridad de los datos sin hacer suposiciones adicionales sobre el comportamiento de los usuarios.

### 2. Correcciones realizadas en el dataset

Durante el proceso de validación y limpieza de datos, se realizaron las siguientes correcciones para asegurar la consistencia de las columnas relacionadas con las fechas y los meses:

#### Correlación entre 'mes_viaje' y 'fecha_viaje'
Se observó que en un gran porcentaje de los datos el valor de **`mes_viaje`** no coincidía con el mes real extraído de la columna **`fecha_viaje`**. Este desajuste fue corregido de la siguiente manera:

- La columna **`mes_viaje`** fue actualizada para reflejar correctamente el mes extraído de la columna **`fecha_viaje`** utilizando la función `dt.month` de pandas.

**Impacto**: Esto asegura que la columna **`mes_viaje`** sea consistente con el mes de la fecha de viaje, mejorando la precisión de los análisis relacionados con las fechas.

#### Correlación entre 'mes_reserva' y 'fecha_reserva'
En algunos registros, el valor de **`mes_reserva`** no coincidía con el mes real extraído de la columna **`fecha_reserva`**. Para corregir esto:

- La columna **`mes_reserva`** fue actualizada para coincidir con el mes extraído de la columna **`fecha_reserva`**, utilizando la misma función `dt.month`.

**Impacto**: Este ajuste asegura que la columna **`mes_reserva`** esté alineada con el mes de la fecha de reserva, facilitando la correcta interpretación y análisis de los datos relacionados con las fechas de reserva.

### 3. Consistencia entre 'estado_reserva' y 'cancelacion_reserva'

Se identificaron algunos registros donde había incoherencias entre las dos columnas:
- En algunos casos, el estado de la reserva era **`cancelada`** pero **`cancelacion_reserva`** era **`no`**.
- En otros casos, el estado de la reserva no era **`cancelada`**, pero **`cancelacion_reserva`** era **`si`**.

#### Acción tomada:
Se corrigieron estos registros, ajustando la columna **`cancelacion_reserva`** para que refleje correctamente el estado de la reserva:
- Si la reserva fue **`cancelada`**, **`cancelacion_reserva`** se cambió a **`si`**.
- Si la reserva no fue **`cancelada`**, **`cancelacion_reserva`** se cambió a **`no`**.

#### Impacto:
Este ajuste garantiza que las columnas **`estado_reserva`** y **`cancelacion_reserva`** sean consistentes, lo que mejora la calidad de los datos para los análisis posteriores.

## Gestión de valores atípicos

In [ ]:
# Crear un dataframe con las columnas numericas
columnas_num = df.select_dtypes(include = np.number).columns.to_list()
lista_eliminar = [ 'mes_reserva', 'mes_viaje']
for col in lista_eliminar:
  columnas_num.remove(col)

In [ ]:
# Observar las medidas estadísticas
df[columnas_num].describe().T

In [ ]:
# Comprobar la presencia de valores atípicos comparando los histogramas con los diagramas de cajas de cada columna
sp.plot_outliers(df, columnas_num )

En base a los análisis visuales realizados, **no se han identificado valores atípicos** en el dataset. Esto indica que los datos parecen estar bien distribuidos y dentro de rangos razonables.

## Gestión de nulos

In [ ]:
df.isnull().sum()

Se observaron algunos **valores nulos** en tres columnas del dataset. Estos valores nulos tienen un **sentido lógico** y **no indican errores** en la recopilación de datos, sino que reflejan casos específicos dentro del contexto del negocio.

**Columna 'comentarios'**: Algunos usuarios pueden no dejar comentarios, ya que no todos los clientes proporcionan retroalimentación sobre su experiencia.

**Columna 'tipo_alojamiento' y 'actividades_reservadas' para 'Solo vuelo'**: Se espera que para 'solo vuelo' no haya valores en las columnas **`tipo_alojamiento`** o **`actividades_reservadas`**.

In [236]:
# Rellenar los nulos con valores fijos
df['tipo_alojamiento'] = df['tipo_alojamiento'].fillna('Sin alojamiento')
df['actividades_reservadas'] = df['actividades_reservadas'].fillna('Sin actividad') 
df['comentarios'] = df['comentarios'].fillna('Sin comentarios')

## Creación de nuevas columnas interesantes para el análisis

In [ ]:
# Crear gasto por persona y gasto por duracion del viaje
df['gasto_por_persona'] = round(df['costo_total'] / df['numero_personas'], 2)
df['gasto_por_duracion'] = round(df['costo_total'] / df['duracion_viaje'],2)
# Crear una columna de rango de edad
bins = [0, 18, 30, 40, 50, 60, 100]
labels = ['0-18', '19-30', '31-40', '41-50', '51-60', '60+']
df['rango_edad'] = pd.cut(df['edad'], bins=bins, labels=labels, right=False)
# Verificar las nuevas columnas
df[['edad','gasto_por_persona', 'gasto_por_duracion', 'rango_edad']].sample(10)

In [ ]:
df.to_csv("../data/halcon_viajes_limpios.csv", index = False) # Guardar los cambios

# Análisis univariado de las variables categóricas, numéricas y temporales
Realizar un análisis univariado para comprender la distribución de cada variable del dataset.
Este paso permite identificar desbalances, valores predominantes y preparar el terreno para el análisis multivariante.

In [ ]:
df = pd.read_csv("../data/halcon_viajes_limpios.csv") # Leer el archivo

## Análisis de las columnas categóricas

In [ ]:
# Seleccionar columnas categóricas
col_cat = ['género', 'ubicación', 'tipo_viajero',
        'tipo_paquete','metodo_pago', 'destino', 'tipo_alojamiento', 'clase_vuelo',
       'actividades_reservadas', 'promocion_aplicada', 'fuente_reserva', 'estado_reserva', 
       'estacionalidad', 'cancelacion_reserva', 'comentarios', 'rango_edad']
# Llamar a la función que analiza las columnas categóricas
sp.subplot_col_cat(df, col_cat)

Conclusiones del análisis de las variables categóricas

1. **género**: La mayoría de los registros corresponden al género 'otro' seguido de 'masculino' y 'femenino' con una distribución igualada.

2. **ubicación**: Distribución uniforme, quedándose méxico una unaminoría significativa.

3. **tipo_viajero**: El tipo de viajero turista representa el doble del de negocios, lo que refleja las tendencias generales del sector de viajes y podría influir en las decisiones de marketing y las promociones de la empresa.

4. **tipo_paquete**: Los paquetes más frecuentes son 'solo_vuelo' y 'vuelo_hotel_tour'.

5. **metodo_pago**: Distribución uniforme en los tres métodos.

6. **destino**: Tokio y Nueva York son los destinos más populares seguido de Paris y Londres, lo cual podría ayudar a la empresa a optimizar sus ofertas y promociones hacia estos destinos.

7. **tipo_alojamiento**: Los tipos más comunes son airbnb y hotel.

8. **clase_vuelo**: La mayoría de las reservas se encuentran en la clase económica, lo que es esperado debido al costo generalmente más bajo. Las clases superiores representan una porción pequeña, lo que puede reflejar una segmentación de clientes más premium.

9. **actividades_reservadas**: Muchas reservas no incluyen actividades adicionales, lo que indica que los viajeros prefieren opciones de viaje más simples o no están interesados en actividades adicionales. En cuanto a los tipos de actividad, no destaca ninguna.

10. **promocion_aplicada**: Se distribuye a partes iguales. Es relevante para entender la efectividad de las campañas promocionales.

11. **fuente_reserva**: Agentes de viajes y la app del móvil son los más utilizados. El sitio web se lleva la minoría.

12. **estado_reserva**: Hay una cantidad muy significativa de reservas canceladas. Casi el triple que confirmadas. El numero de reservas pendientes también es relevante.

13. **estacionalidad**: Se reparten a partes iguales, quedando brevemente por encima la temporada alta.

14. **cancelacion_reserva**: El número de cancelaciones es preocupante, se iguala al número de no cancelaciones. Analizar las razones de las cancelaciones Será crucial para mejorar la retención y entender las dinámicas del comportamiento del cliente.

15. **rango_edad**: El grupo de 19 a 30 años representa la mayoría. El resto refleja una distribución uniforme, quedando por debajo los mayores de 60 años.

16. **comentarios**: Destaca 'problemas con la reserva', además de un número importante de 'no me gustó el servicio'.

## Análisis de las columnas numéricas

In [ ]:
# Seleccionar columnas numéricas
col_num = ['edad', 'antelacion_reserva', 'duracion_viaje', 'numero_personas', 'costo_total', 'numero_noches_estancia', 'calificacion_usuario', 'gasto_por_persona', 'gasto_por_duracion']
# Llamar a la función que analiza las variables numéricas
sp.subplot_col_num(df, col_num)

Conclusiones del análisis de las variables numéricas

1. **edad**: No se observan outliers significativos, y la mayoría de los usuarios están dentro de un rango razonable de edad para viajes.

2. **antelacion_reserva**: Distribución unirforme

3. **duracion_viaje**: Distribución unirforme.

4. **numero_personas**: Distribución unirforme.

5. **costo_total**: Distribución unirforme.

6. **numero_noches_estancia**: Distribución unirforme.

7. **calificacion_usuario**: La distribución es bastante equilibrada, pero con una ligera tendencia hacia calificaciones más altas, lo que sugiere que los usuarios generalmente están satisfechos con los servicios proporcionados.

8. **gasto_por_persona**: La distribución está centrada en valores moderados, aunque también hay algunos registros con un gasto notablemente más alto, que podrían corresponder a viajes de lujo o grupos grandes.
9. **gasto_por_duracion**
 La distribución es más homogénea en comparación con gasto_por_persona, con un rango de valores más estrecho, lo que indica una relación directa entre el costo y la duración del viaje.

## Análisis columnas temporales

In [ ]:
# Obtener el nombre del día a partir de 'fecha_reserva' y 'fecha_viaje'
df['dia_reserva'] = df['fecha_reserva'].dt.day_name()
df['dia_viaje'] = df['fecha_viaje'].dt.day_name()
# Seleccionar variables temporales
col_fecha = ['fecha_reserva', 'fecha_viaje']
# Llamar a la función que analiza las variables temporales
sp.analisis_temporal(df, col_fecha)

Análisis temporal de la fecha de viaje y fecha de reserva

1. **Fecha de Reserva:**
   - **Por año**: 2025
   - **Por mes**: Tendencia decreciente a lo largo del año. La mayoría de las reservas se encuentran en los 4 primeros meses.
   - **Por día**: La gran mayoría reserva los fines de semana. Destacar el miércoles que se lleva el peor día.

2. **Fecha de Viaje:**
   - **Por año**: 2025
   - **Por mes**: A la inversa que la fecha de reserva, tendencia creciente a lo largo del año. Los meses más populares coinciden con las temporadas altas de turismo.
   - **Por día**: Tendencia de viaje los sábados, domingos y jueves, destacando este último. La minoría queda reflejada en el lunes, miércoles y viernes.

Estos patrones podrían ayudar a la empresa a ajustar sus campañas de marketing, promociones y optimización de recursos para aprovechar estos picos.

# Análisis exploratorio de los datos (análisis multivariado)

## Análisis bivariado con la métrica principal
Dado el objetivo del análisis, tomamos como métrica principal la columna 'cancelaciones' y la relacionamos con cada variable del dataset. Este análisis nos permite identificar patrones y comportamientos que afectan las cancelaciones, lo que es clave para reducirlas y aumentar la retención de clientes, objetivo principal de este proyecto.

### 'Cancelaciones' en función de las variables categóricas

In [ ]:
# Lista de columnas categóricas
col_cat = ['género', 'ubicación', 'tipo_viajero',
        'tipo_paquete','metodo_pago', 'destino', 'tipo_alojamiento', 'clase_vuelo',
       'actividades_reservadas', 'promocion_aplicada', 'fuente_reserva', 'estado_reserva', 
       'estacionalidad', 'comentarios', 'rango_edad']
# Llamar a la función para relacionar 'cancelaciones' con cada columna categórica
sp.analisis_cancelaciones_cat(df, 'cancelacion_reserva', col_cat)

Conclusiones análisis de cancelaciones en función de las variables categóricas

1. **género**: El análisis muestra que las cancelaciones están distribuidas de manera equilibrada, con una tendencia ligera a que el femenino produzca más cancelaciones. Esto sugiere que, independientemente del género, las cancelaciones no están fuertemente sesgadas.

2. **ubicación**: Australia tiene una tasa de cancelación más alta. El resto de las ubicaciones muestran una distribución más homogénea. Podría estar relacionado con factores locales, como promociones o estacionalidades.

3. **tipo_viajero**: Los turistas tienen una tasa de cancelación más alta que los viajeros de negocios. Esto podría reflejar un comportamiento más impulsivo o dependiente de factores externos como la disponibilidad de ofertas o condiciones de viaje cambiantes.

4. **tipo_paquete**: El tipo de paquete 'solo_vuelo' tiene casi el doble de cancelaciones que lo que cogen el vuelo con hotel, con actividad o con ambos. Por lo que los que eligen este tipo de paquetes tienden menos a cancelar.

5. **metodo_pago**: Las cancelaciones parecen distribuidas equitativamente entre los métodos de pago disponibles, lo que sugiere que el método de pago no tiene un impacto directo sobre la tasa de cancelación.

6. **destino**: Las cancelaciones son más frecuentes en los destinos de Londres y Roma, lo que podría estar relacionado con factores específicos de estos, como cambios en políticas de cancelación, condiciones de viaje o incluso promociones.

7. **tipo_alojamiento**: Los usuarios que reservan alojamiento en hoteles tienen una mayor tasa de cancelación, posiblemente debido a la mayor flexibilidad en las políticas de cancelación en comparación con otros tipos de alojamiento. Cuando no disponen de alojamiento, son mucho más propensos a cancelar.

8. **clase_vuelo**: Las cancelaciones son más altas en la clase económica, que es la más comúnmente elegida. Esto puede ser indicativo de que los usuarios que eligen vuelos económicos están más propensos a cambiar o cancelar sus planes.

9. **actividades_reservadas**: El tipo de actividad no es determinante en la cancelación. Aunque si no cogen actividad. y por lo tanto tampoco hotel, tienden menos a cancelar.

10. **promocion_aplicada**: La distribución de las cancelaciones es bastante equitativa, lo que indica que las promociones no parecen tener un impacto significativo en la decisión de cancelar una reserva.

11. **fuente_reserva**: Las cancelaciones son ligeramente más frecuentes cuando la reserva se hace a través de agentes de viajes y la app móvil, mientras que el sitio web tiene una tasa de cancelación más baja.  

12. **estado_reserva**: De las reservas que están pendientes, cancela casi la mitad.

13. **estacionalidad**: Las cancelaciones están distribuidas de manera equilibrada entre temporada alta y baja. 

15. **comentarios**: Se observan muchas cancelaciones asociadas con "problemas con la reserva" y "no me gustó el servicio", lo que indica que las cancelaciones están fuertemente relacionadas con la insatisfacción del cliente con el servicio o problemas logísticos durante el proceso de reserva.

### 'Cancelaciones' en función de las variables numéricas

In [ ]:
# Lista de columnas numéricas numéricas
col_num = ['edad', 'antelacion_reserva', 'duracion_viaje', 'numero_personas', 'costo_total', 'numero_noches_estancia', 'calificacion_usuario', 'gasto_por_persona', 'gasto_por_duracion']
# Llamar a la función para relacionar 'cancelaciones' con cada columna numérica
sp.analisis_cancelaciones_num(df, 'cancelacion_reserva', col_num)

Conclusiones de la Relación entre 'cancelaciones' y las variables numéricas

- **edad**: La distribución de la edad entre las cancelaciones y las no cancelaciones parece ser bastante homogénea. Sin embargo, las personas más jóvenes tienden a tener un comportamiento más propenso a cancelar.
  
- **antelacion_reserva**: Los usuarios que reservan mucha antelación, son más propensos a cancelar.
  
- **duracion_viaje**: Cuanto menos dure el viaje, más cancelaciones se producen.
  
- **numero_personas**: No es un factor determinante para las cancelaciones.
  
- **costo_total**: Las reservas de mayor costo parecen tener una menor tasa de cancelación.
  
- **numero_noches_estancia**: No se observa una diferencia significativa en la cantidad de noches de estancia entre las reservas canceladas y no canceladas, lo que sugiere que la duración de la estancia no es un factor determinante.
  
- **calificacion_usuario**: La calificación de los usuarios muestra que las reservas con calificaciones más bajas están asociadas con una mayor tasa de cancelación, lo que podría reflejar insatisfacción con el servicio.
  
- **gasto_por_persona**: No se observa una gran diferencia en el gasto por persona entre las cancelaciones y las no cancelaciones.

- **gasto_por_duracion**: El gasto por duración muestra que no hay una diferencia significativa entre las cancelaciones y no cancelaciones en relación al costo diario de los viajes.

### 'Cancelaciones' en función de las variables temporales

In [ ]:
# Llamar a la función para analizar cancelaciones en función de 'fecha_reserva' y 'fecha_viaje'
sp.analizar_cancelacion_temporal(df, 'cancelacion_reserva', ['mes_reserva', 'mes_viaje', 'dia_reserva', 'dia_viaje'])

No se observa información relevante de las cancelaciones en función del tiempo

## Análisis exploratorio detallado
Realizaremos un análisis multivariado para profundizar en las interacciones entre las variables clave. Este enfoque nos permitirá identificar patrones y relaciones complejas que no son tan evidentes en el análisis hecho anteriormente. Al explorar estas interacciones, podremos obtener una comprensión más detallada de los factores que afectan las cancelaciones y cómo las diferentes variables influyen entre sí.

### Matriz de correlación entre las variables numéricas

In [ ]:
# Lista de columnas numéricas
col_num = ['edad', 'antelacion_reserva', 'duracion_viaje', 'numero_personas', 'costo_total', 'numero_noches_estancia', 'calificacion_usuario', 'gasto_por_persona', 'gasto_por_duracion']
# Llamar a la función de la matriz de correlación
sp.matriz_correlacion(df, col_num)

No se ha extraído ninguna información significativa para el análisis de la matriz de correlación

### ¿Los usuarios de ciertos grupos de edad tienen más probabilidades de cancelar ciertos tipos de paquetes?

In [ ]:
# Llamar a la función que relaciona la edad con el tipo de paquete y con las cancelaciones
sp.edad_tipo_paquete(df)

Para los jóvenes, se podrían ofrecer promociones o incentivos para fomentar el compromiso con las reservas, especialmente para los paquetes más completos.
Para los adultos mayores, mantener el enfoque en paquetes completos (vuelo + hotel + tour) podría ser más efectivo para asegurar menos cancelaciones.

###  ¿Las promociones son más efectivas en ciertos tipos de paquetes?

In [ ]:
# Llamar a la función que analiza la eficiencia de las promociones por tipo de paquete y su relación con las cancelaciones
sp.promociones_tipo_paquete(df)

Las promociones son más efectivas en los paquetes 'vuelo_hotel' y 'vuelo_hotel_tour', donde las cancelaciones son significativamente menores cuando se aplica una promoción.

Las promociones no parecen ser tan efectivas en los paquetes solo_vuelo, ya que los usuarios siguen cancelando con frecuencia incluso cuando reciben un descuento o promoción. Esto podría indicar que las personas que eligen solo vuelos están menos comprometidas con la reserva y más dispuestas a cancelarla, independientemente de las promociones.

(Este análisis puede ayudar a la empresa a decidir en qué tipos de paquetes aplicar promociones de manera más eficaz, especialmente al centrarse en los paquetes más completos para reducir las cancelaciones.)

### ¿Las promociones aplicadas en reservas de bajo costo no están ayudando a reducir las cancelaciones tanto como en las de alto costo?

In [ ]:
# Llamar a la función que analizaa la eficiencia de las promociones según el costo_total y su relación con las cancelaciones
sp.promociones_costo(df)

Las promociones son más eficaces en reservas de alto costo, ya que los usuarios tienden a cancelar menos cuando reciben un descuento sobre una reserva más cara. Esto podría indicar que los descuentos en productos de mayor valor son percibidos como más atractivos y generan mayor compromiso. 
Las promociones en reservas de bajo costo no son lo suficientemente atractivas para evitar cancelaciones. Esto podría ser porque los usuarios ya están más dispuestos a cancelar cuando hacen reservas de bajo costo, independientemente de la promoción recibida.
Este análisis indica que las promociones deberían ser más estratégicamente aplicadas a reservas de alto costo para maximizar su efectividad. Las promociones en reservas de bajo costo pueden necesitar un enfoque diferente o ser acompañadas de otros incentivos.

### ¿Las promociones tienen un impacto diferente en los grupos de edad?

In [ ]:
# Llamar a la función que analiza el impacto de las promociones en la tasa de cancelación según el grupo de edad
sp.promociones_edad(df)

Jóvenes: Las promociones no parecen tener un gran impacto en la reducción de cancelaciones en este grupo. Esto podría ser porque los jóvenes tienden a ser más impredecibles o más influenciados por factores externos, como cambios de planes o impulsos. Además, podría ser necesario ofrecer incentivos adicionales (por ejemplo, mayores descuentos o promociones personalizadas) para aumentar su compromiso.
Adultos mayores: Las promociones son más efectivas en este grupo, ya que parecen reducir significativamente las cancelaciones. Este grupo puede estar más comprometido con sus reservas y, por lo tanto, más receptivo a los descuentos. Las promociones podrían funcionar bien para asegurar la retención en este grupo etario.

### ¿Existe relación entre la fuente de reserva y la calificación del usuario?

In [ ]:
# Llamar a la función que analiza la relación entre la fuente de reserva y la calificación del usuario
sp.reserva_calificacion(df)

En principio, parece que el sitio web no es un factor determinante para la calificación del usuario

### ¿Cuáles son los comentarios más frecuentes en cada fuente de reserva?

In [ ]:
# Llamar a la función que analiza la relación entre los comentarios y la fuente de reserva
sp.comentarios_reserva(df)

Problemas recurrentes en agentes de viaje y app móvil: La frecuencia de comentarios como 'problemas_con_la_reserva' y 'no_me_gustó_el_servicio' en estos canales indica que hay áreas de mejora en la experiencia del usuario, especialmente relacionadas con los agentes de viaje y la app móvil.
Mejorar la experiencia de usuario en la app móvil y con agentes de viaje, ya que estos canales tienen una mayor concentración de comentarios negativos.